## Mini Training for using Accumulated Local Effects (ALE) Plots in R

Accumulated local effects describe **how features influence the prediction of a machine learning model on average**. 

ALE plots are a faster and unbiased alternative to partial dependence plots (PDPs).

**Motivation:** If features of a machine learning model are correlated, the partial dependence plot cannot be trusted.

ALE plots solve this problem by calculating – also **based on the conditional distribution** of the features – differences in predictions instead of averages

##### Load required packages

In [ ]:
library(iml)
library(randomForest)
library(ggplot2)
library(ALEPlot)
library(corrplot)
library(RColorBrewer)

##### Load and prepare data sets

We use the 'Car Insurance Cold Calls' data set from Kaggle (https://www.kaggle.com/kondla/carinsurance), which has already been splitted into training and validation data sets.

In [ ]:
X_train = read.csv("X_train.csv", header = T, sep = ",")
X_valid = read.csv("X_valid.csv", header = T, sep = ",")
y_train = read.csv("y_train.csv", header = F, sep = ",")
y_valid = read.csv("y_valid.csv", header = F, sep = ",")

In [ ]:
colnames(y_train) = colnames(y_valid) = c("Id","CarInsurance")

In [ ]:
train = cbind(X_train, y_train$CarInsurance)
colnames(train)[36] = "CarInsurance"
train = train[,-1]

In [ ]:
train$CarInsurance <- as.factor(as.character(train$CarInsurance))

#### Correlations in the data

As the occurance of correlations in the data makes a difference for the applied methods, we want to have a look at them first.

In [ ]:
M <-cor(X_train)
corrplot(M, type="upper", order="hclust",
         col=brewer.pal(n=8, name="RdYlBu"))

We want to have a closer look at the correlations of the features 'Age' and 'Balance' as we will look at their ALE and PDP Plots below.

For 'Age', we see a stronger positive correlation (between 0.5 and 0.75) to 'Job_retired', as people are more likely to retire with a higher age. In addition, age is correlated to the marital status: We see a negative correlation between 'Age' and 'MartialStatus_single' and a positive correlation between 'Age' and 'MaritalStatus_married', which both makes sense as older people are more likely to be in a relationship or married. 

For 'Balance', no noteworthy correlations can be seen.

Due to these insights, we expect the ALE and Partial Dependence Plot to be quite similar for the 'Balance' feature. As we have seen correlations between age and other features, differences in the ALE Plot and PDP may occur, as ALE Plots are reflecting these correlations whereas PDP do not.

##### Train a Random Forest Model

In [ ]:
rf = randomForest(CarInsurance ~ ., data = train, ntree = 100)

##### Prepare data for iml and create 'predictor'

In [ ]:
X = train[which(names(train) != "CarInsurance")]

### ALE and PDP using the 'iml'-package

In [ ]:
predictor = Predictor$new(rf, data = X, y = train$CarInsurance)

The 'iml' package offers a possibility to **measure how strongly features interact with each other**. The interaction measure regards how much of the variance of $f(x)$ is explained by the interaction.

In [ ]:
interact = Interaction$new(predictor)
plot(interact)

Here, for 'Balance' the overall interaction strength seems to by higher than expected after just looking at all pairwise correlaions and is even higher than for 'Age'. Altogether interactions seem to play a role for both features.

##### Create ALE and PDP plots for feature 'Age'

In [ ]:
ale = FeatureEffect$new(predictor = predictor, feature = "Age" , method = "ale")

plot(ale) + xlim(0,75)

The ALE plot shows a rather high predicted probability of for buying car insurance for young people at the age of 18-20. The probability then continuosly decreases until it reaches it minimum at the age of 30. From then on, the probability increases almost constantly until the age of 60, where people seem to be most likely to buy car insurance.

In [ ]:
pdp = FeatureEffect$new(predictor = predictor, feature = "Age" , method = "pdp")
plot(pdp)

The Partial Dependence Plot shows a shightly. different effect. Here, the predicted probability also decreases from the age of 20 to the age of 30, but then only slightly increases until the age of 55. Then, a massive increase happens until the age of 60, which by far shows the highest probability of buying car insurance. Then, it slightly decreases again.

Considering interactions by using ALE Plots instead of PD Plots indicate that the effect of the age on the predicted probability of buying car insurance rather constantly increases between the ages of 30 and 60 and does not show such a sudden strong increase as the PDP show.

##### Create ALE and PDP plots for feature 'Balance'

In [ ]:
ale = FeatureEffect$new(predictor = predictor, feature = "Balance" , method = "ale")
plot(ale) + scale_x_continuous(limits=c(0,8000))

In [ ]:
pdp = FeatureEffect$new(predictor = predictor, feature = "Balance" , method = "pdp")
plot(pdp) + scale_x_continuous(limits=c(0,8000))

### ALE and PDP Plots using the 'ALEPlot' package

#### Create a prediction function

In [ ]:
yhat <- function(X.model, newdata) as.numeric(predict(X.model, newdata))

#### Create ALE and PDP plots for feature 'Age'

In [ ]:
ALEPlot(X, rf, yhat, J=30, K=50, NA.plot = TRUE)

The ALE plot shows a rather low predicted probability for buying car insurance for young people, which continuosly decreases until it reaches it minimum at the age of 35. From then on, the probability increases almost constantly until the age of 60, with very strong increases between 57 and 65. Overall people around the age of 60 seem to be most likely to buy car insurance.

In [ ]:
PDPlot(X, rf, yhat, J=30, K=50)

The PD plot shows a quite similar trend. The predicted probability for buying car insurance for young people is already rather small, but continuosly decreases until it reaches it minimum at the age of 30. Then the probability remains almost constantly low until the age of 60, where a huge increase happens until the age of around 65, where it reaches its maximum.

The main difference between the PD and the ALE Plot is that for the ALE Plot the initial probability for people at age 18 is already a bit lower and after the decrease until the age of 35 it continuously increases again, whereas in the ALE Plot the probability remains very low between the ages of 30 and 60 and then increases rapidly.

As the age shows a quite large interaction strength, these differences between the ALE and PD Plot are likely a results of interactions, which are covered in the ALE Plot but not in the Partial Dependence Plot.

#### Create ALE and PDP plots for feature 'Balance'

In [ ]:
ALEPlot(X, rf, yhat, J=31, K=50, NA.plot = TRUE)

In [ ]:
PDPlot(X, rf, yhat, J=31, K=50)